In [1]:
# If at any point you're receiving the following TypeError message:
## TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type
# Check the compatibility of your versions of numpy and pandas:
#print(np.__version__)
#print(pd.__version__)

##!pip install numpy==1.16.5 --user
##!pip install pandas==0.25.3 --user

# Import libraries and dependencies
#import panel as pn
#pn.extension('plotly')
#import plotly.express as px
import pandas as pd
#import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv
#import alpaca_trade_api as tradeapi
#from MCForecastTools import MCSimulation
import json, requests

In [2]:
#Read in the data and create a Data Frame. Drop NA as we will only be using the most recent median prices available - 2019-12. Drop unused columns, sort values, and rename '2019-12 column to 'Median Price'
br1_data = pd.read_csv('C:/Users/Myles/Documents/GitHub/bcs-project1/resources/City_MedianRentalPrice_1Bedroom.csv',index_col='RegionName')
br1_df = pd.DataFrame(br1_data).dropna(axis=1).drop(['Unnamed: 0','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11'],axis=1).sort_values('2019-12',ascending=False).rename(columns={'2019-12':'1 BR Median Price'})

br2_data = pd.read_csv('C:/Users/Myles/Documents/GitHub/bcs-project1/resources/City_MedianRentalPrice_2Bedroom.csv',index_col='RegionName')
br2_df = pd.DataFrame(br2_data).dropna(axis=1).drop(['Unnamed: 0','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11'],axis=1).sort_values('2019-12',ascending=False).rename(columns={'2019-12':'2 BR Median Price'})

br3_data = pd.read_csv('C:/Users/Myles/Documents/GitHub/bcs-project1/resources/City_MedianRentalPrice_3bedroom.csv',index_col='RegionName')
br3_df = pd.DataFrame(br3_data).dropna(axis=1).drop(['Unnamed: 0','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11'],axis=1).sort_values('2019-12',ascending=False).rename(columns={'2019-12':'3 BR Median Price'})

br4_data = pd.read_csv('C:/Users/Myles/Documents/GitHub/bcs-project1/resources/City_MedianRentalPrice_4Bedroom.csv',index_col='RegionName')
br4_df = pd.DataFrame(br4_data).dropna(axis=1).drop(['Unnamed: 0','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11'],axis=1).sort_values('2019-12',ascending=False).rename(columns={'2019-12':'4 BR Median Price'})

br5_data = pd.read_csv('C:/Users/Myles/Documents/GitHub/bcs-project1/resources/City_MedianRentalPrice_5BedroomOrMore.csv',index_col='RegionName')
br5_df = pd.DataFrame(br5_data).dropna(axis=1).drop(['Unnamed: 0','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11'],axis=1).sort_values('2019-12',ascending=False).rename(columns={'2019-12':'5+ BR Median Price'})



In [3]:
#Combine the dataframes to include the Median Price for each

brcombined_df = pd.concat(
    [br1_df,br2_df,br3_df,br4_df,br5_df],
    axis=0,
    join="outer"
)
brcombined_df = brcombined_df.sort_index(axis=0).fillna(0)
brcombined_df.head()

,State,Metro,CountyName,SizeRank,1 BR Median Price,2 BR Median Price,3 BR Median Price,4 BR Median Price,2018-12,5+ BR Median Price
RegionName,,,,,,,,,,
Abilene,TX,Abilene,Taylor County,211,565.0,0.0,0.0,0.0,0.0,0.0
Abilene,TX,Abilene,Taylor County,244,0.0,0.0,1295.0,0.0,0.0,0.0
Abilene,TX,Abilene,Taylor County,234,0.0,650.0,0.0,0.0,0.0,0.0
Abington,MA,Boston-Cambridge-Newton,Plymouth County,1190,0.0,2035.0,0.0,0.0,0.0,0.0
Acton,MA,Boston-Cambridge-Newton,Middlesex County,1117,0.0,1795.0,0.0,0.0,0.0,0.0


In [4]:
#Create a pivot table to consolidate the information by Region. We also added states, metro, and CountyName as indices for future analysis options.

br_data_table = pd.pivot_table(
    brcombined_df, 
    index=['State','CountyName','Metro','RegionName'],
    values=['1 BR Median Price','2 BR Median Price','3 BR Median Price','4 BR Median Price','5+ BR Median Price']
)
br_data_table.head()

1 BR Median Price  \
State CountyName                   Metro                 RegionName                      
AK    Anchorage Borough            Anchorage             Anchorage               195.0   
      Fairbanks North Star Borough Fairbanks             Fairbanks                 0.0   
                                                         North Pole                0.0   
      Matanuska Susitna Borough    Anchorage             Wasilla                   0.0   
AL    Baldwin County               Daphne-Fairhope-Foley Daphne                    0.0   

                                                                     2 BR Median Price  \
State CountyName                   Metro                 RegionName                      
AK    Anchorage Borough            Anchorage             Anchorage               240.0   
      Fairbanks North Star Borough Fairbanks             Fairbanks              1200.0   
                                                         North Pole             1285.0   
      Matanuska Susitna Borough    Anchorage             Wasilla                 500.0   
AL    Baldwin County               Daphne-Fairhope-Foley Daphne                    0.0   

                                                                     3 BR Median Price  \
State CountyName                   Metro                 RegionName                      
AK    Anchorage Borough            Anchorage             Anchorage               345.0   
      Fairbanks North Star Borough Fairbanks             Fairbanks                 0.0   
                                                         North Pole                0.0   
      Matanuska Susitna Borough    Anchorage             Wasilla                 697.5   
AL    Baldwin County               Daphne-Fairhope-Foley Daphne                 1352.0   

                                                                     4 BR Median Price  \
State CountyName                   Metro                 RegionName                      
AK    Anchorage Borough            Anchorage             Anchorage               400.0   
      Fairbanks North Star Borough Fairbanks             Fairbanks                 0.0   
                                                         North Pole                0.0   
      Matanuska Susitna Borough    Anchorage             Wasilla                   0.0   
AL    Baldwin County               Daphne-Fairhope-Foley Daphne                    0.0   

                                                                     5+ BR Median Price  
State CountyName                   Metro                 RegionName                      
AK    Anchorage Borough            Anchorage             Anchorage                360.0  
      Fairbanks North Star Borough Fairbanks             Fairbanks                  0.0  
                                                         North Pole                 0.0  
      Matanuska Susitna Borough    Anchorage             Wasilla                    0.0  
AL    Baldwin County               Daphne-Fairhope-Foley Daphne                     0.0

In [5]:
#Set the drive location to the 'resources' folder

cd \Users\Myles\Documents\GitHub\bcs-project1\resources

c:\Users\Myles\Documents\GitHub\bcs-project1\resources


In [6]:
#Save the information to a CSV for purposes of reading it into the program
br_data_table.to_csv('Zillow_Data.csv')